In [1]:
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn import model_selection, metrics
import numpy as np
import pandas as pd

## 1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм".

In [2]:
newsgroups = datasets.fetch_20newsgroups(subset = 'all', categories = ['alt.atheism', 'sci.space'])

In [3]:
data = newsgroups.data
target = newsgroups.target

## 2. Вычислите TF-IDF-признаки для всех текстов. 

Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [4]:
vectorizer = TfidfVectorizer()

In [5]:
data_vect = vectorizer.fit_transform(data)

## 3. Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром (kernel='linear') при помощи кросс-валидации по 5 блокам. 

Укажите параметр random_state=241 и для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).

In [6]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
kf = model_selection.KFold(n_splits = 5, shuffle = True, random_state = 241)
clf = SVC(kernel = 'linear', random_state = 241)
gs = model_selection.GridSearchCV(clf, grid, scoring = 'accuracy', cv = kf)
gs.fit(data_vect, target)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
             error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='linear',
                           max_iter=-1, probability=False, random_state=241,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [7]:
best_C = gs.best_estimator_.C

## 4. Обучите SVM по всей выборке с оптимальным параметром C, найденным на предыдущем шаге.

In [8]:
svc = SVC(C = best_C, kernel = 'linear', random_state = 241)
svc.fit(data_vect, target)
svc.coef_

<1x28382 sparse matrix of type '<class 'numpy.float64'>'
	with 18404 stored elements in Compressed Sparse Row format>

## 5. Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле coef_ у svm.SVC). 

Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [9]:
weights = []
for element in svc.coef_.T:
    weights.append(abs(element[0]))

In [10]:
words = vectorizer.get_feature_names()

In [11]:
important_words_indices = np.argsort(abs(svc.coef_.toarray()[0]))[-10:]

In [12]:
important_words = np.array(words)[important_words_indices]

In [13]:
sorted(important_words)

['atheism',
 'atheists',
 'bible',
 'god',
 'keith',
 'moon',
 'religion',
 'sci',
 'sky',
 'space']